## 분급→선급 수수료 정산 프로그램 : 본 프로그램 사용시 파일명,회사명 확인 후 실행바람
  1. 분급,선급원장 읽기 : 첫행은 반드시 컬럼명이어야  
  2. 분급원장 전처리 : glad 계약 추출, 문홍주건 정제, 지점코드 세팅  
  3. 선급원장 전처리: 해당회사,장기
  4. 증번,회차별 sort  :<span style="color:yellow">미사용</span>
  5. 선급,분급 차이 산출 
  6. 요약,목록 시트 출력  
 [실행파일은 사용하지 말것-- 최신버전 아님]
 본 스크립트는 분급수수료정산.py로 제작되어 실행파일화 함: d:/글로벌/분급정산  

In [ ]:
# import sys
# sys.path.insert(0, 'd:/MyPython/')
# from mypkgs import GladUtil as glad
import pandas as pd
import GladUtil as glad
import time
import os
# 제이엔제이에서 글래드프라임으로 이동건은 지점명을 제이엔제이로 변경-20210701 "
def glad_prime(df_p):
    cond = " 팀 == '프라임2팀' "
    temp = [ '글래드제이엔제이' if x < '20210701' else '글래드프라임' for x in df_p.query(cond)['계약일자'] ]
    cond = df_p['팀'] == '프라임2팀'
    df_p.loc[cond ,'지점'] = temp
    return df_p

def read_source_div():
    '''분급원장 read'''
    df_div = pd.DataFrame()
    df_div = pd.read_excel(inFile1, sheet_name=company,header=0,dtype={'증권번호':object})
    # df_div = df_div[df_div.본부=='MPKJ']
    df_div = df_div[df_div.총괄=='GLAD']
    df_div['계약일자'] = df_div.계약일자.map(lambda x: str(x)[:8])
    # df_div = df_div[~((df_div.사원명=='문홍주') & (df_div.계약일자 < '20180601'))]  #문홍주 구 소속건 삭제
    df_div = glad.BrhConv(df_div,'지점그룹')                               #글래드직속(구,메트로)로지점분리, 직영/지사구분
    df_div = glad_prime(df_div)
    df_div['보험료'].fillna(0, inplace=True)  #결측치 보정
    df_div['수수료'].fillna(0, inplace=True)  #결측치 보정
    return df_div

def read_source_adv(dfW):
    ''' 선급원장 read -- 최초1회만'''
    #선급 원장: 기존 방식의 계산값(글로벌 -> MP)
    start = time.time()  # 시작 시간 저장
    try:
        if len(dfW)==0:
            raise NameError
        val = str(dfW.head(1)['지급년월'][0])
        if val == ym:
            raise NameError("원장년월 변경필요")
    # catch when df1 is None
    except NameError:    
        print('선급원장 start reading...')
        dfW = pd.read_excel(inFile2, header=0, sheet_name='전체',dtype={'계약일자':object})  
    finally:
    #SettingWithCopyWarning 방지를 위해 deep copy사용(https://emilkwak.github.io/pandas-dataframe-settingwithcopywarning)
        df_advance = dfW[(dfW.보험사==company)&(dfW.계약종류=='장기')].copy() 
        df_advance = glad_prime(df_advance)
        df_advance['보험료'].fillna(0, inplace=True)  #결측치 보정
        df_advance['수수료계'].fillna(0, inplace=True)  #결측치 보정
        df_advance = df_advance.sort_values(by=['증권번호','납입회차','상태','보험료']).reset_index()
        print('선급원장 read completion ')
    print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    df_advance = glad.BrhConv(df_advance,'지점그룹')  
    return dfW, df_advance

def make_outfile():
    ''' 회사별 시트 생성'''
    dfT = pd.DataFrame(df_div.groupby('지점')[['수수료']].agg('sum'))

#     df_advance = glad.BrhConv(df_advance,'지점그룹')   
    dfT1 = pd.DataFrame(df_advance.groupby('지점')[['수수료계']].agg('sum'))
    dfT = pd.concat([dfT,dfT1],axis=1)

    dfT.rename(columns = {'수수료':'분급수수료','수수료계':'선급수수료'}, inplace=True)
    dfT['분급수수료'].fillna(0, inplace=True)  #결측치 보정
    dfT['선급수수료'].fillna(0, inplace=True)  #결측치 보정
    dfT['차액(선급-분급)'] = dfT.선급수수료 - dfT.분급수수료
    차액합 = dfT['차액(선급-분급)'].sum()

    직영 = ['글래드레전드','글래드여주','글래드의정부','글래드제이엔제이','글래드씨티엠','글래드프라임-J&J']
    dfT.loc[dfT.index.isin(직영),'%'] = 0 
    dfT.loc[~dfT.index.isin(직영),'%'] = 30
    dfT['MP'] = dfT['%'] * dfT['차액(선급-분급)'] / 100  #귀속분 산출
    s1 = dfT.sum()    #요약값 산출
    s1['%'] = 0       
    dfT.loc['합계'] = s1        #마지막 row에 합계추가
    dfT['이자'] = (dfT['MP'] * 0.035 / 12).astype(int)  #마지막 column열에 이자 3% 적용

    w_mode = 'a' if os.path.isfile(outFile) else 'w'  #파일있으면 append mode
    with pd.ExcelWriter(outFile, mode = w_mode, engine="openpyxl") as writer:
         dfT.to_excel(writer, sheet_name=comp, index=True)
    #     df_div.to_excel(writer, sheet_name=comp+'분급목록',index=False) 
    #     df_advance.to_excel(writer, sheet_name=comp+'선급목록',index=False) 

    #del dfT,s1
    print( 'file created..' + outFile)

    print('{} 선급건수 = {:,.0f} 분급건수 = {:,.0f}'.format(company, len(df_advance),len(df_div)))
    print('선급수수료-원장 = {:,.0f}'.format(df_advance.수수료계.sum() ))
    print('분급수수료-원장 = {:,.0f}'.format(df_div.수수료.sum() ) )
    print('{} 차액 = {:,.0f}'.format(company, 차액합 ) )

def make_summary():
    ''' 당월 요약파일에 전월까지의 요약표를 합치는 작업'''
    from openpyxl import load_workbook
    from openpyxl.styles import Font, Color, Alignment, Border, Side, colors

    def find_row(ws,val):
        # for row in ws['a'].iter_rows():
        #     for cell in row:
        for cell in ws['a']:
            if val == cell.value:
                return cell.row

    def set_border(ws, cell_range):
        thin = Side(border_style="thin", color="000000")
        for row in ws[cell_range]:
            for cell in row:
                cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

    big_blue_text = Font(color=colors.BLUE, size=14)
    double_border_side = Side(border_style="double")
    square_border = Border(top=double_border_side,right=double_border_side,bottom=double_border_side,left=double_border_side)

    print('summary started... ')
    ym_nm = ym[4:]+'월' 
    # 당월 파일 열기 (from 위 작업)
    wb = load_workbook(outFile)
    ws = wb[comp] #wb.active 시트 -> 회사 시트

    # 위치 찾기
    r = find_row(ws,"합계")
    c = [ col.column for col in ws[1] if col.value =='이자'][0]
    이자 = ws.cell(r,c).value
    ws.cell(r,c).font = big_blue_text

    # 전월 파일 열기
    bef_ym = str(int(ym[:4])-1)+'12' if int(ym[4:6])-1==0 else ym[:4]+str(int(ym[4:6])-1).zfill(2)
    befFile = outFile.replace(ym,bef_ym).replace(ym[:4]+'-'+ym[4:], bef_ym[:4]+'-'+bef_ym[4:] )

    try:
        wb_bef = None
        wb_bef = load_workbook(befFile,data_only=True)
        ws_bef = wb_bef[comp]
        # -- 전월 파일에서 
        r = find_row(ws_bef, "이자 정산표")
        # 요약표
        sum_tbl = ws_bef[r:ws_bef.max_row]

        # 행 위치를 같게 만듬
        r_cnt = r - ws.max_row - 1
        ws.insert_rows(ws.max_row + 1,r_cnt)
        ws.cell(ws.max_row+1,1).value = " "
        # for row in ws[ws.max_row+1: ws.max_row + r_cnt]:
        #     row[0].value = " "

        # 요약표를 복사
        # for row in sum_tbl:
        for row in ws_bef.iter_rows(min_row = r,values_only=True):    
            ws.append([c for c in row])
        wb_bef.close()

        # 직전 최종월 column
        r = find_row(ws, "이자 정산표")
        c = [ cell.column for cell in ws[r+3] if cell.value is not None][-1]
        # 직전 최종월 row
        for cell in ws.iter_rows(min_row=r+3, max_row = ws.max_row-1, min_col=c, max_col=c):
            if cell[0].value:
                mx_r = cell[0].row
    except FileNotFoundError:    
        print('전월파일이 없음-분급정산을 시작하는 회사...',company,' -',ws.max_row)
        r= ws.max_row + 2  # next 2 last row
        ws.cell(r, 1).value = "이자 정산표"  
        ws.cell(r+1, 1).value = "수수료"  
        ws.cell(r+1, 2).value = "정산월"
        ws.cell(r+2, 1).value = "지급월"
        ws.cell(r+3, 1).value = "합계"  
        c = 1
        mx_r = r + 2
        print('max row .. ', ws.max_row,' head row.. ', r)
    finally:

        if mx_r == ws.max_row-1:
            ws.insert_rows(ws.max_row)

        # 컬럼이 1개이므로 for 문을 1회만 사용하고 첫컬럼인덱스[0] 참조
        # 전월이자를 next column(당월)로 복사 
        for cell in ws.iter_rows(min_row = r+3, max_row = ws.max_row-1, min_col=c, max_col = c):
            rr = cell[0].row
            ws.cell(rr, c+1).value =  ws.cell(rr, c).value

        # 당월 제목
        ws.cell(r+2, c+1).value = ym_nm 
        ws.cell(mx_r + 1,1).value = ym_nm 
        # 당월 이자
        int_rng = ws.cell(mx_r + 1,c+1)
        int_rng.value = 이자
        # 당월 누적합계 : 수식으로 넣은 후 openpyxl로 읽으면 해당 셀를 dataonly로 읽을때 none나옴
        # a = ws.cell(r+3,c+1).coordinate
        # b = ws.cell(mx_r+1,c+1).coordinate
        # ws.cell(ws.max_row, c+1).value  = f'=sum({a}:{b})'
        sum_val = 0
        for row in ws.iter_rows(min_row=r+3, max_row=mx_r+1, min_col=c+1,max_col=c+1,values_only=True):
            for cell in row:
                 sum_val += cell
        ws.cell(ws.max_row, c+1).value = sum_val

        # style 
        int_rng.font = big_blue_text
        ws.cell(ws.max_row, c+1).font = big_blue_text

        # coordinate는 'a1' 스타일 주소 
        a = ws.cell(r+2,1).coordinate
        b = ws.cell(ws.max_row,c+1).coordinate
        str_rng = f"{a}:{b}"
        set_border(ws, str_rng)

        str_rng = f"A:{str(chr(65+c+1))}"
        for row in ws[str_rng]:
            for cell in row:
                 cell.number_format = '#,##0'

        wb.save(outFile)
        wb.close()
        print(f'{company} : summary completed===========\n')


DICT_COMP = {'롯데':'롯데손보','현대':'현대해상','KB':'KB손보','DB':'DB손보','삼성':'삼성화재'}
DIR_PATH = 'e:/글로벌백업(180329)/04.수수료작업/'

ym ='202109'
# inf_nm1 ='분급이자/'+ ym+'_'+ comp +' 분급 원장.xlsx'
inf_nm1 = '분급이자/'+ ym + '_GLAD수입원장(분급)건별.xlsx'
inf_nm2 = ym + '수수료/'+ ym[0:4]+ ym[4:6]+ '_수입원장.xlsx'
out_nm  = ym + '수수료/'+ ym[0:4]+'-'+ym[4:6]+ ' 선급수수료이자 정산.xlsx'
# print(inf_nm1)

inFile1 = DIR_PATH + inf_nm1                   #분급원장
inFile2 = DIR_PATH + inf_nm2                   #선급원장
outFile = DIR_PATH + out_nm                    #출력파일
dfW = pd.DataFrame()

for comp, company in DICT_COMP.items():
    df_div = read_source_div()
    dfW, df_advance = read_source_adv(dfW)
    make_outfile()
    make_summary()
    
print('<<<<<--- job completed --->>>>>')

#### 방법2[*미사용*]: 건별로 분급,선급원장 비교하여 집계

In [ ]:
''' 분급원장과 선급원장의 건수가 차이나는 경우 -선급건수>분급건수 '''
# 선급원장의 항목을 분급원장에 append함
if len(df_div)!=len(df_advance):
    print('건수차이:  분급원장건수=', len(df_div),
                     '선급원장건수=', len(df_advance) )
    dt = pd.merge(df_advance,df_div,how='outer',on=['증권번호','납입회차','상태','보험료'],indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
    dt2 = pd.merge(df_advance,dt[['증권번호','납입회차','상태','보험료']],how='inner',on=['증권번호','납입회차','상태','보험료'])
    cols1 = list(df_div.columns)   #분급원장컬럼
    cols2 = list(dt2.columns)  #선급원장컬럼
    diff_col = list(set(cols1)-set(cols2))
    for i in diff_col:       #선급원장에 컬럼 추가
        dt2[i] = ""
    dt2['수수료'] = dt2['수수료계']
    dt2['지급'] = dt2['수수료계']
    dt2 = glad.grpConv(dt2,'지점그룹') 
    df_div = df_div.append(dt2[cols1]) #분급원장에 row추가
    print('추가건수= ',len(dt2), ' / 추가금액=' , dt2['수수료'].sum() )
    print('추가  후: 분급원장건수= ',len(df_div), '선급원장건수=' , len(df_advance) )
else: print('건수일치...= ', len(df_div))

''' 분급원장 읽기 및 결과파일 생성 '''
#concat를 하기 위해 index순번을 재부여
df_div = df_div.sort_values(by=['증권번호','납입회차','상태','보험료']).reset_index()
df_div = pd.concat([df_div, df_advance.수수료계], axis=1)  #차례대로 합침
df_div.rename(columns = {'수수료계':'선급수수료','수수료':'분급수수료'}, inplace=True)
df_div['차액(선급-분급)'] = df_div.선급수수료 - df_div.분급수수료

#요약시트 생성
dfT = pd.DataFrame(df_div.groupby('지점')[['선급수수료','분급수수료','차액(선급-분급)']].agg('sum'))
직영 = ['글래드레전드','글래드여주','글래드의정부','글래드제이엔제이','글래드씨티엠']
dfT.loc[dfT.index.isin(직영),'%'] = 0 
dfT.loc[~dfT.index.isin(직영),'%'] = 30
dfT['MP'] = dfT['%'] * dfT['차액(선급-분급)'] / 100  #귀속분 산출
s1 = dfT.sum()    #요약값 산출
s1['%'] = 0       
dfT.loc['합계'] = s1        #마지막 row에 합계추가
dfT['이자'] = (dfT['MP'] * 0.035 / 12).astype(int)  #마지막 column열에 이자 3% 적용

w_mode = 'a' if os.path.isfile(outFile) else 'w'  #파일있으면 append mode
with pd.ExcelWriter(outFile, mode = w_mode, engine="openpyxl") as writer:
    dfT.to_excel(writer, sheet_name=comp,index=True)
    df_div.to_excel(writer, sheet_name=comp+'목록',index=False) 

#del dfT,s1
print( 'created..' + outFile)

### 정상여부 검증
if  (df_div.선급수수료.sum() - df_advance.수수료계.sum()) != 0:
     print('오류발생xxxxxxxxxxxxxxxxxxxxxxxxxx')
elif  df_div.선급수수료.sum() != s1.선급수수료:        
     print('오류발생-분급누락건 점검요~ xxxxxxx')
        ##분급원장에 없는 증번은 수입원장에서 분급원장으로 건을 복사 후 실행요망
else: print('정상종료-------------------------')

print('선급수수료-matched = {:,.0f}'.format(s1.선급수수료) )
print('선급수수료-원장 = {:,.0f}'.format(df_advance.수수료계.sum() ))
print('분급수수료-원장 = {:,.0f}'.format(df_div.분급수수료.sum() ) )
print('차액 = {:,.0f}'.format(df_div['차액(선급-분급)'].sum() ) )

### 만약 오류가 있으면 
  선급원장에 증번+회차+상태가 중복이면서 분급원장에 중복수만큼 있지않은경우
  1. 아래작업으로 증번확인 후
  2. 그 아래 추가작업2 실시
  3. 마지막으로 '처음'~'분급원장 읽기 및 결과파일 생성' 직전까지
  4. 아래 추가 작업3 실시
  4. '분급원장 읽기 및 결과파일 생성' 작업부터 실시

In [ ]:
#증권번호별 건수를 구한다음 건수 불일치 증번 display
counts=pd.DataFrame(df_div.groupby(['증권번호'])['증권번호'].agg('count')).fillna(method='ffill')
counts2=pd.DataFrame(df_advance.groupby(['증권번호'])['증권번호'].agg('count')).fillna(method='ffill')
counts2['건수일치']=counts2.증권번호==counts.증권번호
counts2[counts2.건수일치==False]

In [ ]:
#추가할 건에 대해 확인 후 작업하여 df에 추가
dt3=df_advance[(df_advance.증권번호=='299046164900') & (df_advance.지급구분=='환수')]
cols1=list(df_div.columns)   #분급원장컬럼
cols2=list(dt3.columns)  #선급원장컬럼
diff_col=list(set(cols1)-set(cols2))
for i in diff_col:       #선급원장에 컬럼 추가
    dt3[i]=""
dt3['수수료']=dt3['수수료계']
dt3['지급']=dt3['수수료계']

In [ ]:
df=df_div.append(dt3[cols1]) #분급원장에 row추가

#### 작업 후 차이가 나는 경우 찾는 방법 2

In [ ]:
df_div_cnt=df_div.groupby(['증권번호'])['증권번호'].count().reset_index(name='cnt')
df_advance_cnt=df_advance.groupby(['증권번호'])['증권번호'].count().reset_index(name='cnt')
df_cnt=pd.concat([df_advance_cnt,df_div_cnt],join='inner',axis=1)
df_cnt.columns=['증권번호','선급건수','증','분급건수']
ply=df_cnt.loc[df_cnt.선급건수!=df_cnt.분급건수,'증권번호'].values.tolist()
df_advance[df_advance.증권번호.isin(ply)]

In [ ]:
df_div[df_div.분급수수료!=df_div.지급+df_div.환수][['증권번호','분급수수료','지급','환수']]

In [ ]:
#df_div[df_div.증권번호.isin(df3_kj.index)]
df_T=df_advance[['증권번호','납입회차','수수료계']]
df=df_div.join(df_T.set_index(['증권번호','납입회차'])['수수료계'], on=['증권번호','납입회차'])